In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import necessary libraries

In [ ]:
from scipy import stats
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sales_df = pd.read_csv('/kaggle/input/sales-dataset/sales_data.csv')

In [ ]:
sales_df.head()

Columns in the Dataset


* **Product_ID**: Unique identifier for each product sold. Randomly generated for practice purposes.
* **Sale_Date**: The date when the sale occurred. Randomly selected from the year 2023.
* **Sales_Rep**: The sales representative responsible for the transaction. The dataset includes five random sales representatives (Alice, Bob, Charlie, David, Eve).
* **Region**: The region where the sale took place. The possible regions are North, South, East, and West.
* **Sales_Amount**: The total sales amount for the transaction, including discounts if any. Values range from 100 to 10,000 (in currency units).
* **Quantity_Sold**: The number of units sold in that transaction, randomly generated between 1 and 50.
* **Product_Category**: The category of the product sold. Categories include Electronics, Furniture, Clothing, and Food.
* **Unit_Cost**: The cost per unit of the product sold, randomly generated between 50 and 5000 currency units.
* **Unit_Price**: The selling price per unit of the product, calculated to be higher than the unit cost.
* **Customer_Type**: Indicates whether the customer is a New or Returning customer.
* **Discount**: The discount applied to the sale, randomly chosen between 0% and 30%.
* **Payment_Method**: The method of payment used by the customer (e.g., Credit Card, Cash, Bank Transfer).
* **Sales_Channel**: The channel through which the sale occurred. Either Online or Retail.
* **Region_and_Sales_Rep**: A combined column that pairs the region and sales representative for easier tracking.


In [ ]:
sales_df.isnull().sum()

## Explore Data

In [ ]:
sales_df.describe()

In [ ]:
sales_df.info()

## Statistical Analysis

In [ ]:
mean_of_Price = sales_df['Unit_Price'].mean()
mean_of_Price

In [ ]:
mean_of_Discount = sales_df['Discount'].mean()
Discount_mean = mean_of_Discount*100 
Discount_mean

As example if we take a price of 20$ of a electronic accessories, the Discount will be : **20*0.15**

In [ ]:
customer_counts = sales_df['Customer_Type'].value_counts()

count_of_returning = customer_counts.get('Returning', 0)
count_of_new = customer_counts.get('New', 0) 

print(f'Number of Returning Customers: {count_of_returning}')
print(f'Number of New Customers: {count_of_new}')


In [ ]:
sales_df.columns

In [ ]:
Categs = sales_df['Product_Category'].unique()
Categs[0]

### A T-test for two categories

In [ ]:
Categ_per_sales = []
for i in range(len(Categs)):
    Categ_per_sales.append(sales_df[sales_df['Product_Category'] == Categs[i]]['Sales_Amount'])

t_stat, p_value = stats.ttest_ind(Categ_per_sales[0], Categ_per_sales[1])

print(f'T-statistic: {t_stat}, P-value: {p_value} ')
#result interpretation

# Interpret the results
if p_value < 0.05:
    print(f"There is a significant difference in sales amounts between {Categs[0]} and {Categs[1]}.")
else:
    print(f"There is no significant difference in sales amounts between {Categs[0]} and {Categs[1]}.")

### Let's perform a pairwise t-tests


In [ ]:
results = []
for (i, j) in itertools.combinations(range(len(Categ_per_sales)), 2):
    t_stat, p_value = stats.ttest_ind(Categ_per_sales[i], Categ_per_sales[j])
    results.append((Categs[i], Categs[j], t_stat, p_value))
    
#let's print results
for cat1, cat2, t_stat, p_value in results:
    print(f'Test between {cat1} and {cat2}: T-statistic = {t_stat}, P-value = {p_value}')
    if p_value < 0.05:
        print(f"There is a significant difference in sales amounts between {cat1} and {cat2}.")
    else:
        print(f"There is no significant difference in sales amounts between {cat1} and {cat2}.")

In [ ]:
New_costumers = sales_df[sales_df['Customer_Type'] == 'New']['Sales_Amount']
Retruning_costumers = sales_df[sales_df['Customer_Type'] == 'Returning']['Sales_Amount']

t_stat, p_value = stats.ttest_ind(New_costumers, Retruning_costumers)

print(f'T-statistic: {t_stat}, P-value: {p_value}')

# Interpret the results
if p_value < 0.05:
    print("There is a significant difference in sales amounts between new and returning customers.")
else:
    print("There is no significant difference in sales amounts between new and returning customers.")

In [ ]:
sales_df.columns

In [ ]:
regions = sales_df.Region.unique()
region_type = []
for i in range(len(regions)):
    region_type.append(sales_df[sales_df['Region'] == regions[i]]['Sales_Amount'])
    
means  = [np.mean(sales) for sales in region_type]

plt.bar(regions,means, color='orange')
plt.ylabel('Means Sales')
plt.xticks(rotation = 45)
plt.title('Means Sales Per Region')

## Visualisation

In [ ]:
means = [np.mean(sales) for sales in Categ_per_sales]
errors = [np.std(sales) / np.sqrt(len(sales)) for sales in Categ_per_sales]
plt.bar(Categs, means, yerr = errors)
plt.xlabel('Categories')
plt.ylabel('Mean Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.title('Mean Sales by Category with Error Bars')

### Aggregate Sales by Region and Category

In [ ]:
sales_df.head()

# Group by Region and Category, then sum the Sales
grouped_sales = sales_df.groupby(['Region', 'Product_Category'])['Sales_Amount'].sum().reset_index()
grouped_sales

In [ ]:
# find the most Bought category based on each country 
most_bought = grouped_sales.loc[grouped_sales.groupby('Region')['Sales_Amount'].idxmax()]
most_bought


In [ ]:
plt.bar(most_bought['Region'], most_bought['Sales_Amount'])
plt.xlabel('regions')
plt.ylabel('Sales amount per category')
plt.title('Most Bought Category Sales Amount by Region')

for i, (region, sales, category) in enumerate(zip(most_bought['Region'], most_bought['Sales_Amount'], most_bought['Product_Category'])):
    plt.text(i, sales + 10, f'{category}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
sales_df.head()
sales_channel = sales_df.Sales_Channel.unique()
sales_channel

In [ ]:
online = sales_df['Sales_Channel'].value_counts().get('Online', 0)
offline = sales_df['Sales_Channel'].value_counts().get('Retail', 0)

online, offline
fig, ax = plt.subplots()
ax.pie([online, offline], labels=['Online', 'Retail'], autopct='%1.1f%%', startangle=90)

## Trending sales

In [ ]:
sales_df.head()

In [ ]:
# convert Date to datetime
#sales_df['Sale_Date'] = pd.to_datetime(sales_df['Sale_Date'])
plt.figure(figsize=(15, 5))
sns.lineplot(data=sales_df, x=sales_df.Sale_Date, y='Sales_Amount', marker = 'o')
plt.title('Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Sales Amount')
plt.xticks(rotation=45)  
plt.grid()
plt.tight_layout() 
plt.show()

In [ ]:
sales_df['Sale_Date'].dt.month
current_date = sales_df['Sale_Date'].max()
last_two_months=current_date - pd.DateOffset(months=2)
current_date

filtered_sales = sales_df[(sales_df['Sale_Date'] >= last_two_months) & (sales_df['Sale_Date'] <= current_date)]

plt.figure(figsize=(15, 5))
sns.lineplot(data=filtered_sales, x=sales_df.Sale_Date, y='Sales_Amount', marker = 'o')
plt.title('Sales Trend Over Time for Last Month')
plt.xlabel('Date')
plt.ylabel('Sales Amount')
plt.xticks(rotation=45)  
plt.grid()
plt.tight_layout() 
plt.show()

# Feature engineering

In [ ]:
# One-Hot encoding
#sales_df = pd.get_dummies(sales_df, columns = ['Sales_Channel'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#Label Encoding 
Products_categories = label_encoder.fit_transform(sales_df['Product_Category'])

In [ ]:
sales_df.Sale_Date

In [ ]:
#sales_df['Year'] = sales_df['Sale_Date'].dt.year
#sales_df['Month'] = sales_df['Sale_Date'].dt.month
#sales_df['Day'] = sales_df['Sale_Date'].dt.day
sales_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
#sales_df['Product_Category'] = labelencoder.fit_transform(sales_df[['Product_Category']]) 
#sales_df['Region'] = labelencoder.fit_transform(sales_df[['Region']]) 



In [ ]:
sales_df.head()
months = sales_df.Month.unique()
months[0]

In [ ]:
most_month_sales = []
for month in months:
    total_sales_month = sales_df[sales_df['Month']== month]['Sales_Amount'].sum() 
    most_month_sales.append(total_sales_month )
    
most_month_sales

## Scaling Features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sales_df[['Sales_Amount']] = scaler.fit_transform(sales_df[['Sales_Amount']])

### 1. **Sales Prediction**
   - **Objective**: Predict future sales amounts based on historical data.
   - **Model**: 
     - **Linear Regression**: A good starting point for predicting continuous values like sales amounts.
     - **Random Forest Regressor**: Can capture non-linear relationships and interactions between features.
     - **XGBoost**: An efficient and powerful gradient boosting framework that often performs well in regression tasks.

### 2. **Customer Segmentation**
   - **Objective**: Segment customers into different groups based on their purchasing behavior.
   - **Model**:
     - **K-Means Clustering**: A popular clustering algorithm to group customers based on features like `Sales_Amount`, `Quantity_Sold`, and `Discount`.
     - **Hierarchical Clustering**: Another clustering method that can provide insights into customer segments.

### 3. **Sales Channel Effectiveness**
   - **Objective**: Analyze which sales channel (Online vs. Retail) is more effective in terms of sales amount.
   - **Model**:
     - **Logistic Regression**: If you want to predict the likelihood of a sale being made through a particular channel based on other features.
     - **Decision Trees**: To visualize and understand the decision-making process behind sales channel effectiveness.

### 4. **Sales Representative Performance**
   - **Objective**: Evaluate the performance of different sales representatives.
   - **Model**:
     - **ANOVA (Analysis of Variance)**: To compare the sales amounts across different sales representatives.
     - **Random Forest Classifier**: If you want to classify sales representatives based on their performance metrics.

### 5. **Discount Impact Analysis**
   - **Objective**: Understand how discounts affect sales amounts.
   - **Model**:
     - **Multiple Linear Regression**: To analyze the relationship between `Discount` and `Sales_Amount`, controlling for other variables.
     - **Regression Trees**: To capture non-linear relationships between discounts and sales.

### 6. **Churn Prediction**
   - **Objective**: Predict whether a customer is likely to return based on their purchase history.
   - **Model**:
     - **Logistic Regression**: To predict the probability of a customer being new or returning based on their purchase behavior.
     - **Random Forest Classifier**: To improve prediction accuracy and handle feature interactions.

### Data Preprocessing Steps
Before training any model, consider the following preprocessing steps:
- **Encoding Categorical Variables**: Convert categorical variables like `Sales_Rep`, `Region`, `Product_Category`, `Customer_Type`, `Payment_Method`, and `Sales_Channel` into numerical format using techniques like one-hot encoding or label encoding.
- **Handling Missing Values**: Check for and handle any missing values in the dataset.
- **Feature Scaling**: Normalize or standardize numerical features like `Sales_Amount`, `Unit_Cost`, and `Unit_Price` if necessary.
- **Feature Engineering**: Create new features if needed, such as total profit (calculated as `Sales_Amount - (Quantity_Sold * Unit_Cost)`).


In [ ]:
sales_df.head()

In [ ]:
sales_df.head()

In [ ]:
X = sales_df.drop(columns=['Sales_Amount', 'Product_ID', 'Sale_Date', 'Region_and_Sales_Rep', 'Customer_Type','Payment_Method' , 'Sales_Rep'])
y = sales_df['Sales_Amount']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

Linear_model = LinearRegression()
Linear_model.fit(x_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

### Make Predictions


In [ ]:
y_pred_linear = Linear_model.predict(x_test)
y_pred_rf = rf_model.predict(x_test)

### Evaluate the Model

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Let's Evaluate linear regression
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f'Mean squared error: {mse_linear}')
print(f'Score R2: {r2_linear}')

In [ ]:
# Let's evaluate random forest regression

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f'Mean squared error: {mse_rf}')
print(f'Score R2: {r2_rf}')

 ### Visualisation of prediction

In [ ]:
### plt.scatter(y_test, y_pred_linear, label = 'linear regression predictions')
plt.scatter(y_test, y_pred_linear, label = 'Linear regression predictions')
plt.scatter(y_test, y_pred_rf, label = 'Rnadom forest predictions')
plt.plot([y.min(), y.max()], [y.min(), y.max()], "k--", lw=2)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
sales_df.columns

In [ ]:
X = sales_df.drop(columns=['Product_ID', 'Sale_Date', 'Sales_Rep', 'Sales_Amount','Customer_Type', 'Discount', 'Payment_Method', 'Region_and_Sales_Rep', 'Month', 'Day'])
y = sales_df['Sales_Amount']
X

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)

model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2_score = r2_score(y_test, y_pred)
print(f'Mean squared Error: {mse}')
print(f'Score R2: {r2_score}')